### Convolutional Neural Networks - Transfer Learning

KIAS CAC Summer School 2021

Dates: 2021-7-12

Author: Yung-Kyun Noh

Department of Computer Science, Hanyang University / School of Computational Sciences, KIAS


In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

In [ ]:
# VGG16 network pre-trained on the ImageNet dataset
from tensorflow.keras.applications import VGG16

model = VGG16(weights="imagenet")
model.summary()

In [ ]:
from tensorflow import keras

base_model = keras.applications.VGG16(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(224, 224, 3),
    include_top=False)

base_model.summary()

In [ ]:
base_model.trainable = False

In [ ]:
inputs = keras.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, name='predictions')(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=True), metrics=[keras.metrics.BinaryAccuracy()])

https://www.kaggle.com/ankitachoudhury01/covid-patients-chest-xray

In [ ]:
# data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        samplewise_center=True,  # set each sample mean to 0
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False) # we don't expect Bo to be upside-down so we will not flip vertically

In [ ]:
# load and iterate training dataset
train_it = datagen.flow_from_directory('./COVID/train/', 
                                       target_size=(224, 224), 
                                       color_mode='rgb', 
                                       class_mode='binary', 
                                       batch_size=8)
# load and iterate validation dataset
valid_it = datagen.flow_from_directory('./COVID/test/', 
                                      target_size=(224, 224), 
                                      color_mode='rgb', 
                                      class_mode='binary', 
                                      batch_size=8)

In [ ]:
model.fit(train_it, steps_per_epoch=12, validation_data=valid_it, validation_steps=4, epochs=20)

In [ ]:
# fine tunning
base_model.trainable = True

# reduce learning rate
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate = .00001),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])

In [ ]:
model.fit(train_it, steps_per_epoch=10, validation_data=valid_it, validation_steps=4, epochs=20)

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input
import matplotlib.pyplot as plt

# _img = load_img("./COVID/test/COVID_PATIENTS/covid_77.jpg",target_size=(224,224))
_img = load_img("./COVID/test/NORMAL_PATIENTS/normal_70.jpg",target_size=(224,224))
plt.imshow(_img)
plt.show()

In [ ]:
import numpy as np
img = np.array(_img).reshape(-1, 224, 224, 3)
y_pred = model.predict(img)
print(y_pred)

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)